In [1]:
import numpy as np

import matplotlib.pyplot as plt

import pandas as pd

from matplotlib import gridspec

from sklearn.preprocessing import scale

from tqdm import tqdm_notebook as tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader

from torch.utils.data import Dataset

from sklearn.utils import shuffle
from scipy.special import softmax

from PIL import Image

from PIL import ImageDraw

from skimage import morphology

import pickle

In [2]:
from MixtureLib.Mixture import MixtureEM
from MixtureLib.LocalModels import EachModelLinear, EachModel
from MixtureLib.HyperModels import HyperExpertNN, HyperModelDirichlet

from MixtureLib.Regularizers import RegularizeModel

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'